# E2B Code Sandbox with Anthropic
Execute AI-generated Python code in a secure sandbox environment.

In [1]:
"""E2B Code Sandbox Demo."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from anthropic import Anthropic

# if sys.platform == 'win32':
#     sys.stdout.reconfigure(encoding='utf-8')

## Initialize Sandbox and Upload Dataset

In [2]:
sbx = Sandbox.create()
print("Sandbox created successfully")

with open("data/titanic.csv", "rb") as f:
    dataset_path_in_sandbox = sbx.files.write("data/titanic.csv", f)

print(f"Dataset uploaded to: {dataset_path_in_sandbox.path}")

Sandbox created successfully
Dataset uploaded to: /home/user/data/titanic.csv


## Code Execution Function

In [3]:
def run_ai_generated_code(ai_generated_code: str):
    """Execute AI-generated code in sandbox and save chart outputs."""
    print('Running code in sandbox....')
    execution = sbx.run_code(ai_generated_code)
    print('Code execution finished!')

    if execution.error:
        print('AI-generated code had an error.')
        print(f"Error: {execution.error.name}")
        print(f"Value: {execution.error.value}")
        print(f"Traceback: {execution.error.traceback}")
        return

    result_idx = 0
    for result in execution.results:
        if result.png:
            filename = f'chart-{result_idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            print(f'Chart saved to {filename}')
            result_idx += 1

## Create Prompt for AI

In [ ]:
prompt = f"""
I have a CSV file about movies. It has about 10k rows. It's saved in the sandbox at {dataset_path_in_sandbox.path}.
These are the columns:
- 'id': number, id of the movie
- 'original_language': string like "eng", "es", "ko", etc
- 'original_title': string that's name of the movie in the original language
- 'overview': string about the movie
- 'popularity': float, from 0 to 9137.939. It's not normalized at all and there are outliers
- 'release_date': date in the format yyyy-mm-dd
- 'title': string that's the name of the movie in english
- 'vote_average': float number between 0 and 10 that's representing viewers voting average
- 'vote_count': int for how many viewers voted

Write Python code that creates a line chart showing how vote_average changed over the years.
Do NOT print or explore the data. Just create the visualization directly.

CRITICAL: Your code MUST end with this exact line to display the plot:
display(plt.gcf())"""

print("Prompt created")

## Generate and Execute Code with Anthropic

In [ ]:
anthropic = Anthropic()

print("Waiting for model response...")

msg = anthropic.messages.create(
    model='claude-haiku-4-5-20251001',
    max_tokens=1024,
    messages=[{"role": "user", "content": prompt}],
    tools=[
        {
            "name": "run_python_code",
            "description": "Run Python code",
            "input_schema": {
                "type": "object",
                "properties": {
                    "code": {"type": "string", "description": "The Python code to run"},
                },
                "required": ["code"]
            }
        }
    ]
)

for content_block in msg.content:
    if content_block.type == "tool_use":
        if content_block.name == "run_python_code":
            code = content_block.input["code"]
            print("\nGenerated code:")
            print(code)
            print("\nExecuting in sandbox...")
            run_ai_generated_code(code)

## Cleanup

In [ ]:
sbx.close()
print("Sandbox closed")